# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
import polars as pl
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
    df['signal_prices'] = 0
    df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [7]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [9]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
    df['exit_prices'] = 0
    df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [10]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [11]:
@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio


In [12]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [13]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [14]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [15]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [16]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[-(7 * 60 * 24 * 52):]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

In [17]:
# def generate_52w_data(data_path):
    
#     df = pl.read_csv(data_path)
#     df = df.with_columns(pl.col('datetime').str.to_datetime())
#     # df = df.slice(-(7 * 60 * 24 * 52))
#     df = df.sort('datetime', descending=False)

#     return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [18]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min_all.csv')
df_52w = generate_52w_data(data_path)
df_52w = df_52w.iloc[(252000+50400): (252000+2*50400)]
df_52w.reset_index(drop=True, inplace=True)
df_52w.head()

,datetime,btc_open,btc_high,btc_low,btc_close,btc_volume,6e_open,6e_high,6e_low,6e_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2024-05-29 18:14:00,67400.12,67406.84,67390.27,67390.27,5.607836,1.08235,1.08235,1.08235,1.08235,...,105.023438,105.023438,105.023438,105.023438,27.0,107.828125,107.843750,107.828125,107.84375,2.0
1,2024-05-29 18:15:00,67390.27,67403.82,67369.42,67389.63,6.103289,1.08235,1.08235,1.08230,1.08230,...,105.023438,105.023438,105.007812,105.007812,768.0,107.828125,107.828125,107.812500,107.81250,1269.0
2,2024-05-29 18:16:00,67389.63,67410.62,67368.18,67368.19,7.088498,1.08225,1.08230,1.08225,1.08230,...,105.007812,105.007812,105.000000,105.000000,465.0,107.812500,107.812500,107.812500,107.81250,327.0
3,2024-05-29 18:17:00,67368.19,67368.25,67346.04,67365.39,3.110999,1.08225,1.08230,1.08225,1.08230,...,105.000000,105.007812,105.000000,105.007812,50.0,107.812500,107.812500,107.812500,107.81250,3.0
4,2024-05-29 18:18:00,67365.39,67372.64,67342.87,67357.31,3.343183,1.08225,1.08230,1.08225,1.08225,...,105.000000,105.007812,105.000000,105.000000,589.0,107.796875,107.812500,107.796875,107.81250,651.0


## Loading strategies generated

In [19]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_10_29_121047_958800_24976_958800/\ge_results.csv')

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,exit_buy,sell,exit_sell,fitness
0,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-11153.937044
1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-11153.937044
2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-11153.937044
3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-11153.937044
4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-11153.937044
...,...,...,...,...,...
16327,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] != 1),((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] != 1),-1.545069
16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 5),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 5),-1.531616
16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] <= 6),((numba_indicators.adx(high=price_data['btc_hi...,(price_data['day_of_week'][MAX_LAG:] <= 6),-1.516913
16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['day_of_week'][MAX_LAG:] < 2),((numba_indicators.exponential_moving_average(...,(price_data['day_of_week'][MAX_LAG:] < 2),-1.509397


## Running the tests over the strategies

In [20]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    buy_exit_txt = row.exit_buy
    sell_signal_txt = row.sell
    sell_exit_txt = row.exit_sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_buy_sell_pnl, get_drawdowns, get_pnl, get_lag
from fitness.performance.helper_func import trading_signals_buy, trading_signals_sell, change_exit
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, buy_exit_idxs = trading_signals_buy(buy_signal={buy_signal_txt}, exit_signal={buy_exit_txt})
sell_idxs, sell_exit_idxs = trading_signals_sell(sell_signal={sell_signal_txt}, exit_signal={sell_exit_txt})
if len(buy_idxs) == 0 or len(sell_idxs) == 0 or len(buy_exit_idxs) == 0 or len(sell_exit_idxs) == 0:
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs = change_exit(buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs)
    if len(buy_idxs) == 0 or len(sell_idxs) == 0 or len(buy_exit_idxs) == 0 or len(sell_exit_idxs) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
    else:
        buy_idxs = np.array(buy_idxs)
        sell_idxs = np.array(sell_idxs)
        open_prices = price_data['btc_open']
        # pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
        buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
        buy_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_exit_idxs)]
        sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
        sell_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_exit_idxs)]
        buy_arr = get_pnl(buy_exit_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        sell_arr = get_pnl(sell_exit_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        all_arr = merge_buy_sell_pnl(buy_idxs, sell_idxs, buy_arr, sell_arr)
        total_pnl = buy_pnl + sell_pnl
        equity_curve_arr = np.cumsum(all_arr)
        drawdowns = get_drawdowns(equity_curve_arr)
        if len(drawdowns[drawdowns!=0]) != 0:
            avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
            fitness = total_pnl / avg_drawdown
        elif total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
            fitness = -9999999
            avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        # df = df_52w.iloc[idx:idx+bars_per_5week, :]
        # df.reset_index(drop=True, inplace=True)
        df = df_52w.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        price_data['day_of_week'] = (df['datetime'].dt.dayofweek + 1).values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=False)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

16332it [5:36:05,  1.23s/it]


In [40]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [41]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,12.039730,0.000000,14.771600,0.0
1,strategy2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,12.039730,0.000000,14.769615,0.0
2,strategy3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,12.039730,0.011907,14.766526,0.0
3,strategy4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,12.039730,0.000000,14.771600,0.0
4,strategy5,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,12.039730,0.000000,14.769615,0.0
...,...,...,...,...,...,...,...
16327,strategy16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,45.467684,68.207832,77.393263,0.0
16328,strategy16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,0.004106,41.558708,58.603983,0.0
16329,strategy16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.adx(high=price_data['btc_hi...,44.331652,19.867694,22.435436,0.0
16330,strategy16331,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,40.638644,45.187537,79.975393,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [42]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
131,strategy132,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,74.008432,60.025799,74.800071,0.0
132,strategy133,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,73.997789,60.025799,74.798587,0.0
160,strategy161,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,74.011091,60.025799,74.802556,0.0
162,strategy163,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,74.010028,60.025799,74.798087,0.0
246,strategy247,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,81.326629,78.648048,78.643508,0.0
...,...,...,...,...,...,...,...
16176,strategy16177,((numba_indicators.moving_average(prices=price...,(numba_indicators.adx(high=price_data['btc_hig...,69.606721,68.112285,91.410421,0.0
16258,strategy16259,((price_data['btc_close'][MAX_LAG:] > get_lag(...,(numba_indicators.moving_average(numba_indicat...,68.409909,77.859029,77.858297,0.0
16260,strategy16261,((numba_indicators.moving_average(prices=price...,(numba_indicators.moving_average(numba_indicat...,100.000000,100.000000,100.000000,0.0
16277,strategy16278,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,63.067203,79.312540,79.308079,0.0


## Showing exit testing results

In [43]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,47.269516,0.0
1,strategy2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,46.133238,0.0
2,strategy3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,46.755824,0.0
3,strategy4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,44.841750,0.0
4,strategy5,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,46.036313,0.0
...,...,...,...,...,...,...,...
16327,strategy16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,59.730518,57.664596,45.123137,0.0
16328,strategy16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.764253,57.664596,46.508166,0.0
16329,strategy16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.adx(high=price_data['btc_hi...,59.732503,57.658641,46.365134,0.0
16330,strategy16331,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.702736,57.686429,45.412334,0.0


### Showing strategies that have passed entry testing

In [44]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
131,strategy132,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.734487,57.664596,45.915107,0.0
132,strategy133,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.732503,57.670550,45.807373,0.0
160,strategy161,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.728534,57.664596,47.200286,0.0
162,strategy163,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.726549,57.664596,45.489542,0.0
246,strategy247,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.714643,57.664596,46.173383,0.0
...,...,...,...,...,...,...,...
16176,strategy16177,((numba_indicators.moving_average(prices=price...,(numba_indicators.adx(high=price_data['btc_hig...,59.724565,57.664596,45.537167,0.0
16258,strategy16259,((price_data['btc_close'][MAX_LAG:] > get_lag(...,(numba_indicators.moving_average(numba_indicat...,59.726549,57.664596,45.584063,0.0
16260,strategy16261,((numba_indicators.moving_average(prices=price...,(numba_indicators.moving_average(numba_indicat...,59.730518,57.664596,46.055399,0.0
16277,strategy16278,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,59.730518,57.664596,46.181320,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [ ]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working


## Showing core testing results

In [46]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,57.142857,0.0
1,strategy2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,57.142857,0.0
2,strategy3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,62.500000,0.0
3,strategy4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,57.142857,0.0
4,strategy5,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,57.142857,0.0
...,...,...,...,...,...
16327,strategy16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,57.142857,0.0
16328,strategy16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,11.111111,0.0
16329,strategy16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.adx(high=price_data['btc_hi...,60.000000,0.0
16330,strategy16331,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,30.000000,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [47]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
131,strategy132,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,75.000000,0.0
132,strategy133,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,75.000000,0.0
160,strategy161,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,75.000000,0.0
162,strategy163,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,75.000000,0.0
246,strategy247,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,100.000000,0.0
...,...,...,...,...,...
15865,strategy15866,((numba_indicators.moving_average(prices=price...,((numba_indicators.exponential_moving_average(...,76.000000,0.0
15923,strategy15924,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,71.428571,0.0
16010,strategy16011,(numba_indicators.moving_average(numba_indicat...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",90.000000,0.0
16258,strategy16259,((price_data['btc_close'][MAX_LAG:] > get_lag(...,(numba_indicators.moving_average(numba_indicat...,66.666667,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [48]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working


## Showing performance

In [49]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
0,strategy1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,6.0,-8527.329961,-2.436380,14559.939425,400.981295,5.0,-0.585671,-93.143507,-76.054975,0.0
1,strategy2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,6.0,-8527.329961,-2.436380,14559.939425,400.981295,5.0,-0.585671,-93.143507,-76.054975,0.0
2,strategy3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,7.0,-8505.291477,-2.430083,14559.939425,400.981295,6.0,-0.584157,-87.693832,-66.376112,0.0
3,strategy4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,6.0,-8527.329961,-2.436380,14559.939425,400.981295,5.0,-0.585671,-93.143507,-76.054975,0.0
4,strategy5,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,6.0,-8527.329961,-2.436380,14559.939425,400.981295,5.0,-0.585671,-93.143507,-76.054975,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16327,strategy16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,6.0,-7862.259500,-2.246360,17990.189375,423.841627,5.0,-0.437030,-108.725847,-80.587316,0.0
16328,strategy16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,8.0,-17183.373710,-4.909535,13893.168606,2401.235751,4.0,-1.236822,-658.754437,-474.387005,0.0
16329,strategy16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.adx(high=price_data['btc_hi...,9.0,-832.596220,-0.237885,522.612373,658.934836,2.0,-1.593143,-21.115035,-16.319275,0.0
16330,strategy16331,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,9.0,-26745.849768,-7.641671,15585.228707,462.269432,4.0,-1.716102,-323.155223,-273.273739,0.0


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [50]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
131,strategy132,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,7.0,23436.665603,6.696190,7.310278e+03,194.558574,4.0,3.205988e+00,249.356892,215.482390,0.0
132,strategy133,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,7.0,19142.074470,5.469164,7.310278e+03,194.558574,4.0,2.618515e+00,205.494196,175.996877,0.0
160,strategy161,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,7.0,24273.493390,6.935284,7.310278e+03,194.558574,4.0,3.320461e+00,257.231200,223.176387,0.0
162,strategy163,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,7.0,22740.773209,6.497364,7.310278e+03,194.558574,4.0,3.110794e+00,242.356783,209.084186,0.0
246,strategy247,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,5.0,5264.502920,1.504144,-9.999999e+06,92.562203,4.0,-9.999999e+06,558.903437,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15769,strategy15770,((numba_indicators.exponential_moving_average(...,((numba_indicators.bull_bar_tail_rolling(close...,19.0,12353.648892,3.529614,1.244975e+03,2396.301890,8.0,9.922806e+00,157.626546,250.432687,0.0
15865,strategy15866,((numba_indicators.moving_average(prices=price...,((numba_indicators.exponential_moving_average(...,24.0,28860.754886,8.245930,8.217222e+02,120.197187,23.0,3.512228e+01,230.552444,569.962382,0.0
16010,strategy16011,(numba_indicators.moving_average(numba_indicat...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",9.0,15683.444840,4.480984,2.384734e+03,9687.391918,2.0,6.576602e+00,405.491060,476.792273,0.0
16258,strategy16259,((price_data['btc_close'][MAX_LAG:] > get_lag(...,(numba_indicators.moving_average(numba_indicat...,5.0,135.623175,0.038749,2.757673e+03,254.808216,1.0,4.918029e-02,4.417796,3.515542,0.0


### Showing performance for strategies that has passed all tests and have positive ROI

In [51]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working


In [52]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [53]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [54]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [55]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,105.173554,4.0,-7665.927048,-2.190265,-0.020825,0.3732,0.0
1,strategy2,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,100.980757,4.0,-8420.001348,-2.405715,-0.023823,0.3663,0.0
2,strategy3,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,122.320891,5.0,-7705.101738,-2.201458,-0.017997,0.3694,0.0
3,strategy4,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,100.401323,4.0,-7572.689190,-2.163625,-0.021550,0.3752,0.0
4,strategy5,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,100.847476,4.0,-7478.820992,-2.136806,-0.021188,0.3697,0.0
...,...,...,...,...,...,...,...,...,...,...
16327,strategy16328,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.moving_average(prices=price...,42.964713,3.0,-7096.990209,-2.027711,-0.047195,0.3593,0.0
16328,strategy16329,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,0.000000,8.0,-17035.262647,-4.867218,-inf,0.0001,0.0
16329,strategy16330,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.adx(high=price_data['btc_hi...,106.064624,5.0,53.137861,0.015182,0.000143,0.5037,0.0
16330,strategy16331,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,0.000000,9.0,-26578.242199,-7.593783,-inf,0.0599,0.0


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [56]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
131,strategy132,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,41.315898,2.0,24004.033670,6.858295,0.165997,0.8478,0.0
132,strategy133,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,45.899138,2.0,19725.800211,5.635943,0.122790,0.7966,0.0
160,strategy161,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,38.319982,2.0,25593.822138,7.312521,0.190828,0.8595,0.0
162,strategy163,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,41.592796,2.0,23146.955699,6.613416,0.159004,0.8372,0.0
246,strategy247,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",((numba_indicators.exponential_moving_average(...,0.000000,0.0,5116.644043,1.461898,inf,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
15769,strategy15770,((numba_indicators.exponential_moving_average(...,((numba_indicators.bull_bar_tail_rolling(close...,68.240555,7.0,11514.450336,3.289843,0.048209,0.8376,0.0
15865,strategy15866,((numba_indicators.moving_average(prices=price...,((numba_indicators.exponential_moving_average(...,19.904771,4.0,27353.818225,7.815377,0.392638,0.9905,0.0
16010,strategy16011,(numba_indicators.moving_average(numba_indicat...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",15.475429,1.0,15219.993301,4.348570,0.280998,0.9782,0.0
16258,strategy16259,((price_data['btc_close'][MAX_LAG:] > get_lag(...,(numba_indicators.moving_average(numba_indicat...,78.480838,3.0,249.353324,0.071244,0.000908,0.5359,0.0


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [57]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working


In [39]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_last.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_last.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_last.csv', index=False)
final_perf_df.to_csv('testing_results/perf_last.csv', index=False)
final_mc_df.to_csv('testing_results/mc_last.csv', index=False)